<a href="https://colab.research.google.com/github/anko191/Python_Kaggle/blob/master/lightgbm_xgboost/lightgbm_digit_recognizer_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys, os
sys.path.append(os.pardir)
%pwd

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('--'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('--')
test = pd.read_csv('--')
sub = pd.read_csv('--')

from sklearn.model_selection import train_test_split
T_train, T_valid = train_test_split(train, test_size = 0.2, shuffle=True)
y_train, X_train = T_train.iloc[:,0], T_train.iloc[:,1:]
y_valid, X_valid = T_valid.iloc[:,0], T_valid.iloc[:,1:]
X_test = test

import matplotlib.pyplot as plt
X_train = X_train.values
y_train = y_train.values
X_valid = X_valid.values
y_valid = y_valid.values
X_test = X_test.values
X_train,X_valid = X_train/255, X_valid/255
X_train.shape
X_valid.shape

(8400, 784)

In [ ]:
import lightgbm as lgb
lgb.LGBMClassifier(max_bin=255, min_data_in_leaf=20, min_sum_hessian_in_leaf=0.001,
               num_iterations=100, save_binary=False)

LGBMClassifier(max_bin=255, min_data_in_leaf=20, min_sum_hessian_in_leaf=0.001,
               num_iterations=100, save_binary=False)

In [ ]:
train_data = lgb.Dataset(X_train, label = y_train)
valid_data = lgb.Dataset(X_valid, label = y_valid, reference=train_data)
params = {
    'task':'train',
    'boosting_type':'gbdt',
    'objective':'multiclass',
    'num_class':10,
    'verbose':0,
}
gbm = lgb.train(
    params,
    train_data,
    valid_sets = valid_data,
    num_boost_round=100,
    verbose_eval=5,
)

[5]	valid_0's multi_logloss: 1.28053
[10]	valid_0's multi_logloss: 0.87798
[15]	valid_0's multi_logloss: 0.646354
[20]	valid_0's multi_logloss: 0.49882
[25]	valid_0's multi_logloss: 0.401073
[30]	valid_0's multi_logloss: 0.33211
[35]	valid_0's multi_logloss: 0.282697
[40]	valid_0's multi_logloss: 0.246893
[45]	valid_0's multi_logloss: 0.219228
[50]	valid_0's multi_logloss: 0.197791
[55]	valid_0's multi_logloss: 0.180916
[60]	valid_0's multi_logloss: 0.16747
[65]	valid_0's multi_logloss: 0.156897
[70]	valid_0's multi_logloss: 0.14806
[75]	valid_0's multi_logloss: 0.140428
[80]	valid_0's multi_logloss: 0.134517
[85]	valid_0's multi_logloss: 0.129129
[90]	valid_0's multi_logloss: 0.124539
[95]	valid_0's multi_logloss: 0.120199
[100]	valid_0's multi_logloss: 0.116641


In [ ]:
preds = gbm.predict(X_valid)
y_pred = []
for x in preds:
    y_pred.append(np.argmax(x))
from sklearn.metrics import accuracy_score

In [ ]:
y_valid

array([8, 0, 7, ..., 0, 3, 0], dtype=int64)

In [ ]:
np.array(y_pred)

array([8, 0, 7, ..., 0, 3, 0], dtype=int64)

In [ ]:
print(accuracy_score(y_valid, y_pred))

0.965952380952381


In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 4, shuffle =True)
kf

KFold(n_splits=4, random_state=None, shuffle=True)

In [ ]:
train = pd.read_csv('--')
test = pd.read_csv('--')
sub = pd.read_csv('--')

for tr_idx, va_idx in kf.split(train):
    T_train, T_valid = train.iloc[tr_idx,:], train.iloc[va_idx,:]
    y_train, X_train = T_train.iloc[:,0], T_train.iloc[:,1:]
    y_valid, X_valid = T_valid.iloc[:,0], T_valid.iloc[:,1:]
    X_train = X_train.values
    y_train = y_train.values
    X_valid = X_valid.values
    y_valid = y_valid.values
    X_test = test
    X_test = X_test.values
    X_train,X_valid = X_train/255, X_valid/255
    train_data = lgb.Dataset(X_train, label = y_train)
    valid_data = lgb.Dataset(X_valid, label = y_valid, reference=train_data)
    params = {
        'task':'train',
        'boosting_type':'gbdt',
        'objective':'multiclass',
        'num_class':10,
        'verbose':0,
    }
    gbm = lgb.train(
        params,
        train_data,
        valid_sets = valid_data,
        num_boost_round=100,
        verbose_eval=5,
    )
    preds = gbm.predict(X_valid)
    y_pred = []
    for x in preds:
        y_pred.append(np.argmax(x))
    from sklearn.metrics import accuracy_score
    np.array(y_pred)
    print(accuracy_score(y_valid, y_pred))

[5]	valid_0's multi_logloss: 1.2728
[10]	valid_0's multi_logloss: 0.870071
[15]	valid_0's multi_logloss: 0.63694
[20]	valid_0's multi_logloss: 0.49009
[25]	valid_0's multi_logloss: 0.393308
[30]	valid_0's multi_logloss: 0.323331
[35]	valid_0's multi_logloss: 0.273624
[40]	valid_0's multi_logloss: 0.236815
[45]	valid_0's multi_logloss: 0.209617
[50]	valid_0's multi_logloss: 0.188694
[55]	valid_0's multi_logloss: 0.171966
[60]	valid_0's multi_logloss: 0.158346
[65]	valid_0's multi_logloss: 0.147501
[70]	valid_0's multi_logloss: 0.138896
[75]	valid_0's multi_logloss: 0.130986
[80]	valid_0's multi_logloss: 0.124493
[85]	valid_0's multi_logloss: 0.118958
[90]	valid_0's multi_logloss: 0.113991
[95]	valid_0's multi_logloss: 0.110021
[100]	valid_0's multi_logloss: 0.10651
0.9689523809523809
[5]	valid_0's multi_logloss: 1.27263
[10]	valid_0's multi_logloss: 0.868949
[15]	valid_0's multi_logloss: 0.637313
[20]	valid_0's multi_logloss: 0.490574
[25]	valid_0's multi_logloss: 0.39323
[30]	valid_0's

In [ ]:
preds = gbm.predict(X_test)
y_pred = []
for x in preds:
    y_pred.append(np.argmax(x))
sub['Label'] = list(map(int,y_pred))
sub.to_csv('submission.csv', index = False)